In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn. feature_extraction. text import TfidfTransformer , TfidfVectorizer , CountVectorizer
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, roc_auc_score
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk. corpus import stopwords
from nltk. stem import PorterStemmer
from nltk. stem. wordnet import WordNetLemmatizer
from gensim. models import Word2Vec, KeyedVectors
import pickle
import scipy as sp
from tqdm import tqdm
import os
from sklearn. cross_validation import train_test_split
from sklearn. neighbors import KNeighborsClassifier
from sklearn. cross_validation import cross_val_score
from collections import Counter
from sklearn import cross_validation
from sklearn. preprocessing import normalize, StandardScaler
from sklearn import datasets, neighbors

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import tree




from scipy.sparse import load_npz

path = "C:/Python/Assignments/Preprocessing/"

def opn(name):
    with open(path + name , "rb") as file:
        return pickle.load(file)


with open(path +"X_test.txt", "rb") as file:
    X_test = pickle.load(file)
with open(path +"X_train.txt", "rb") as file:
    X_train = pickle.load(file)
with open(path +"X_cv.txt", "rb") as file:
    X_cv = pickle.load(file)
    
with open(path +"Y_test.txt", "rb") as file:
    Y_test = pickle.load(file)
with open(path +"Y_train.txt", "rb") as file:
    Y_train = pickle.load(file)
with open(path +"Y_cv.txt", "rb") as file:
    Y_cv = pickle.load(file)

with open(path + "X_cv_sum.txt", "rb") as file:
    X_cv_sum = pickle.load(file)
with open(path + "X_train_sum.txt", "rb") as file:
    X_train_sum = pickle.load(file)
with open(path + "X_test_sum.txt", "rb") as file:
    X_test_sum = pickle.load(file)
    

C:\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# RF function using X

In [2]:
import xgboost as xgb
def DT(train1,test1,cv1):

    sc = StandardScaler(with_mean=False)
    train = sc.fit_transform(train1)
    test = sc.transform(test1)
    
    clf = xgb.XGBClassifier(max_depth=8,n_estimators=500)
    clf.fit(train,Y_train)
    pred_prob = clf.predict_proba(test)
    print("The AUC value for test data is ",roc_auc_score(Y_test, pred_prob[:,1]))

## Calculate Helpfulness percentage to add as a feature

In [3]:
# Get the helpfulness data
with open(path + "final_sorted.txt", "rb") as file:
    sorted_data = pickle.load(file)
my_final = sorted_data[:100000]
my_final['Helpful_percent'] = (my_final['HelpfulnessNumerator'] / my_final['HelpfulnessDenominator'])*100
my_final['Helpful_percent'].fillna(0, inplace=True)
from sklearn.model_selection import train_test_split
x = my_final['Helpful_percent'].values
y = my_final['Score']

# split the data set into train and test
X_1, X_test_perc, y_1, Y_test1 = cross_validation.train_test_split(x, y, test_size=0.3, random_state=0)

# split the train data set into cross validation train and cross validation test
X_train_perc, X_cv_perc, Y_train1, Y_cv1 = cross_validation.train_test_split(X_1, y_1, test_size=0.3,random_state=0)

# https://stackoverflow.com/questions/41927781/adding-pandas-columns-to-a-sparse-matrix/41948136
from scipy.sparse import hstack
def addmat(col,csr_mat):
    '''
    Add a new column to sparse matrix with the number of words in each entry of col
    '''
    leng = []
    for i in col:
        leng.append(len(i.split(" ")))
    temp = hstack((csr_mat, pd.Series(leng)[:,None]))
    return temp.tocsr()


# TfIdf

In [4]:
#Adding review text and review summary to create a new data
train = X_train + X_train_sum
cv = X_cv + X_cv_sum
test = X_test + X_test_sum

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
train_vect = tf_idf_vect.fit_transform(train)
cv_vect = tf_idf_vect.transform(cv)
ts_vect = tf_idf_vect.transform(test)
print("With review text and summary both\n\n")

DT(train_vect,ts_vect,cv_vect)
print("With review text length\n\n")
# Adding the number of words each review text as a new feature
tr1 = addmat(X_train, train_vect)
cv1 = addmat(X_cv, cv_vect)
ts1 = addmat(X_test, ts_vect)
DT(tr1,ts1,cv1)
print("\n\nWith review summary length\n\n")
# Adding the number of words each review summary as a new feature
tr1 = addmat(X_train_sum, tr1)
cv1 = addmat(X_cv_sum, cv1)
ts1 = addmat(X_test_sum, ts1)

DT(tr1,ts1,cv1)
print("\n\nWith helpfullness percentage\n\n")
# Adding Helpfulness Percentage as a new feature
tr1 = hstack((tr1, pd.Series(X_train_perc)[:,None]))
cv1 = hstack((cv1, pd.Series(X_cv_perc)[:,None]))
ts1 = hstack((ts1, pd.Series(X_test_perc)[:,None]))
DT(tr1,ts1,cv1)

With review text and summary both


The AUC value for test data is  0.9419054106873062
With review text length


The AUC value for test data is  0.94151889638981


With review summary length


The AUC value for test data is  0.9423949163768021


With helpfullness percentage


The AUC value for test data is  0.9560893887162235


# Conclusion

In [14]:
# Please compare all your models using Prettytable library
from prettytable import from_csv
with open("added.csv", "r") as fp: 
    x = from_csv(fp)

print(x)
fp.close()

+------------+--------------------------------+------+
| Vectorizer |          New Feature           | AUC  |
+------------+--------------------------------+------+
|   TFIDF    |        Only Review Text        | 0.94 |
|   TFIDF    | Review Summary along with Text | 0.94 |
|   TFIDF    |       Review text length       | 0.94 |
|   TFIDF    |     Review summary length      | 0.94 |
|   TFIDF    |     Helpfulness Percentage     | 0.96 |
+------------+--------------------------------+------+


1. I have used XGBOOST with tfidf as this gave me the best AUC score before, So I am trying to improve upon that.
2. There is improvement on the model after adding the helpful percentage as a new feature. The AUC score got bumped upto 96% from 94%.
3. There may be other features which will improve the model more than this.